# Zenith vs PyTorch: 1GB Dataset Benchmark

**Large-scale benchmark with synthetic 1GB dataset**

In [ ]:
!pip install pyarrow torch --quiet
print("Dependencies installed")

## Generate 1GB Synthetic Dataset

In [ ]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import os

print("Generating 1GB synthetic dataset...")

# 100k samples x 32x32x3 x float32 = ~1.2GB
NUM_SAMPLES = 100000
IMG_SHAPE = (3, 32, 32)  # CHW format
NUM_CLASSES = 10

print(f"Samples: {NUM_SAMPLES:,}")
print(f"Image shape: {IMG_SHAPE}")
print(f"Expected size: ~{NUM_SAMPLES * np.prod(IMG_SHAPE) * 4 / 1e9:.1f} GB")

# Generate in chunks to avoid OOM
chunk_size = 10000
all_images = []
all_labels = []

for i in range(0, NUM_SAMPLES, chunk_size):
    n = min(chunk_size, NUM_SAMPLES - i)
    images = np.random.rand(n, *IMG_SHAPE).astype(np.float32)
    labels = np.random.randint(0, NUM_CLASSES, n)
    all_images.append(images)
    all_labels.append(labels)
    print(f"  Generated {i+n:,}/{NUM_SAMPLES:,}")

images = np.concatenate(all_images)
labels = np.concatenate(all_labels)

print(f"\nTotal: {images.shape}")
print(f"Memory: {images.nbytes / 1e9:.2f} GB")

In [ ]:
# Save as Parquet
print("Saving to Parquet...")

table = pa.table({
    'image': pa.array([img.tobytes() for img in images]),
    'label': pa.array(labels, type=pa.int64())
})

pq.write_table(table, 'synthetic_1gb.parquet', compression='snappy')

size_mb = os.path.getsize('synthetic_1gb.parquet') / (1024**2)
print(f"Saved: synthetic_1gb.parquet ({size_mb:.0f} MB)")

# Free memory
del all_images, all_labels
import gc
gc.collect()

## Define Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.reshape(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

## Benchmark: Zenith DataLoader

In [ ]:
import time
import torch.optim as optim

class ZenithDataLoader:
    def __init__(self, path, batch_size=256):
        self.table = pq.read_table(path, memory_map=True)
        self.batch_size = batch_size
        self.num_rows = self.table.num_rows
    
    def __iter__(self):
        indices = np.arange(self.num_rows)
        np.random.shuffle(indices)
        for start in range(0, self.num_rows, self.batch_size):
            end = min(start + self.batch_size, self.num_rows)
            yield self.table.take(indices[start:end])
    
    def __len__(self):
        return (self.num_rows + self.batch_size - 1) // self.batch_size

print("="*50)
print("ZENITH DATALOADER - 1GB DATASET")
print("="*50)

zenith_loader = ZenithDataLoader('synthetic_1gb.parquet', batch_size=256)
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

zenith_times = []
for epoch in range(3):
    model.train()
    start = time.time()
    total_loss = 0
    
    for batch in zenith_loader:
        img_bytes = batch.column('image').to_pylist()
        labels_np = batch.column('label').to_numpy()
        
        images_np = np.array([np.frombuffer(b, dtype=np.float32).reshape(3, 32, 32) 
                              for b in img_bytes])
        
        images = torch.from_numpy(images_np).to(device)
        labels = torch.from_numpy(labels_np).long().to(device)
        
        optimizer.zero_grad()
        loss = criterion(model(images), labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    elapsed = time.time() - start
    zenith_times.append(elapsed)
    print(f"Epoch {epoch+1}: Loss={total_loss/len(zenith_loader):.4f}, Time={elapsed:.2f}s")

zenith_avg = sum(zenith_times[1:]) / len(zenith_times[1:])
print(f"\nZenith avg: {zenith_avg:.2f}s")

## Benchmark: PyTorch DataLoader

In [ ]:
from torch.utils.data import DataLoader as TorchDataLoader, TensorDataset

print("="*50)
print("PYTORCH DATALOADER - 1GB DATASET")
print("="*50)

# Load into memory (PyTorch needs this)
print("Loading data into memory for PyTorch...")
images_tensor = torch.from_numpy(images)
labels_tensor = torch.from_numpy(labels).long()

torch_loader = TorchDataLoader(
    TensorDataset(images_tensor, labels_tensor),
    batch_size=256,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

model = SimpleCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

torch_times = []
for epoch in range(3):
    model.train()
    start = time.time()
    total_loss = 0
    
    for images_batch, labels_batch in torch_loader:
        images_batch = images_batch.to(device)
        labels_batch = labels_batch.to(device)
        
        optimizer.zero_grad()
        loss = criterion(model(images_batch), labels_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    elapsed = time.time() - start
    torch_times.append(elapsed)
    print(f"Epoch {epoch+1}: Loss={total_loss/len(torch_loader):.4f}, Time={elapsed:.2f}s")

torch_avg = sum(torch_times[1:]) / len(torch_times[1:])
print(f"\nPyTorch avg: {torch_avg:.2f}s")

## Final Results

In [ ]:
print("\n" + "="*50)
print("BENCHMARK RESULTS - 1GB SYNTHETIC DATASET")
print("="*50)
print(f"Dataset: {NUM_SAMPLES:,} samples, ~1GB")
print(f"Batch size: 256")
print(f"Device: {device}")
print("-"*50)
print(f"Zenith DataLoader:  {zenith_avg:.2f}s per epoch")
print(f"PyTorch DataLoader: {torch_avg:.2f}s per epoch")
print("-"*50)

if zenith_avg < torch_avg:
    speedup = torch_avg / zenith_avg
    print(f"Result: Zenith is {speedup:.2f}x faster")
else:
    speedup = zenith_avg / torch_avg
    print(f"Result: PyTorch is {speedup:.2f}x faster")